In [ ]:
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability
from pyincore.analyses.buildingfunctionality import BuildingFunctionality
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.epfdamage import EpfDamage
from pyincore import Dataset

In [ ]:
# define client
client = IncoreClient()

## Chaining with Galveston Building Damage and EPF Pole damage

### Building damage

In [ ]:
galveston_bldg_dmg = BuildingDamage(client)

fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping("602c381a1d85547cdc9f0675"))
galveston_bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

galveston_bldg_dmg.load_remote_input_dataset("buildings", "60354b6c123b4036e6837ef7")
galveston_bldg_dmg.set_parameter("fragility_key", "Hurricane SurgeLevel and WaveHeight Fragility ID Code")
galveston_bldg_dmg.set_parameter("result_name", "Galveston_bldg_dmg_result")
galveston_bldg_dmg.set_parameter("hazard_type", "hurricane")
galveston_bldg_dmg.set_parameter("hazard_id", "5fa5a228b6429615aeea4410")
galveston_bldg_dmg.set_parameter("num_cpu", 4)
galveston_bldg_dmg.run_analysis()

### Monte Carlo buildings failure probability

In [ ]:
galveston_bldg_dmg_result = galveston_bldg_dmg.get_output_dataset('ds_result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", galveston_bldg_dmg_result)
mc.set_parameter("num_cpu", 4)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
mc.set_parameter("failure_state_keys", ["moderate", "heavy", "complete"])

mc.set_parameter("result_name", "gal_mcs_buildings_samples")
mc.run_analysis()

gal_bldg_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
gal_bldg_damage_mcs_samples_df = gal_bldg_damage_mcs_samples.get_dataframe_from_csv()
gal_bldg_damage_mcs_samples_df.head()

### Pole Damage

In [ ]:
epf_dmg_hurricane_galveston = EpfDamage(client)
epf_dmg_hurricane_galveston.load_remote_input_dataset("epfs", "62fc000f88470b319561b58d")

# Load fragility mapping
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping("62fac92ecef2881193f22613"))
epf_dmg_hurricane_galveston.set_input_dataset('dfr3_mapping_set', mapping_set)

epf_dmg_hurricane_galveston.set_parameter("result_name", "galveston_hurricane_epf_damage")
epf_dmg_hurricane_galveston.set_parameter("hazard_type", "hurricane")
epf_dmg_hurricane_galveston.set_parameter("hazard_id", "5fa5a228b6429615aeea4410")
epf_dmg_hurricane_galveston.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
epf_dmg_hurricane_galveston.set_parameter("num_cpu", 8)

# Run Analysis
epf_dmg_hurricane_galveston.run_analysis()

### MC poles failure probability

In [ ]:
gal_pole_dmg_result = epf_dmg_hurricane_galveston.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", gal_pole_dmg_result)
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["DS_0","DS_1","DS_2","DS_3","DS_4"])
mc.set_parameter("failure_state_keys", ["DS_1","DS_2","DS_3","DS_4"])

mc.set_parameter("result_name", "gal_mcs_poles_samples")
mc.run_analysis()

gal_pole_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
gal_pole_damage_mcs_samples_df = gal_pole_damage_mcs_samples.get_dataframe_from_csv()
gal_pole_damage_mcs_samples_df.head()

### Chaining with building functionality analysis

In [ ]:
gal_bldg_func = BuildingFunctionality(client)
    
gal_bldg_func.set_input_dataset("building_damage_mcs_samples", gal_bldg_damage_mcs_samples)
gal_bldg_func.set_input_dataset("poles_damage_mcs_samples", gal_pole_damage_mcs_samples)

gal_interdependency_dictionary = Dataset.from_file("/Users/cwang138/Documents/INCORE-2.0/reformatted_closest_poles_and_substations_to_bldgs.json",
                                                   data_type="incore:buildingInterdependencyDict")
gal_bldg_func.load_remote_input_dataset("interdependency_dictionary", "5dcf4a34b9219ca5e4118312")

gal_bldg_func.set_parameter("result_name", "gal_mcs_functionality_probability")
gal_bldg_func.run_analysis()

gal_bldg_func_samples_df = gal_bldg_func.get_output_dataset('functionality_samples').get_dataframe_from_csv()
gal_bldg_func_samples_df.head()

## Chaining with Joplin Building Damage

### Tornado

In [ ]:
# The simulated EF-5 tornado shows geographical locations and the range
# of wind speed of tornado hazard in Joplin.
hazard_type = "tornado"
hazard_id = "5d07cbeab9219c065b080930"

### Building damage

In [ ]:
bldg_dmg = BuildingDamage(client)

mapping_id = "5d8942dfb9219c068ea795ea"  # Create a mapping to assign tornado fragilities to 19 building archetypes.
fragility_service = FragilityService(client)        # loading fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

bldg_dataset_id = "5dbc8478b9219c06dd242c0d"  # Load the Joplin building dataset, which is a package of GIS files.

bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
result_name = "Joplin_bldg_dmg_result"
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 4)  # Define the result name, etc., and choose the number of CPU locally
                                      # to run the model parallelly.

In [ ]:
bldg_dmg.run_analysis()

### Monte Carlo buildings failure probability

In [ ]:
building_dmg_result = bldg_dmg.get_output_dataset('ds_result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", building_dmg_result)
mc.set_parameter("num_cpu", 4)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
mc.set_parameter("failure_state_keys", ["moderate", "heavy", "complete"])

mc.set_parameter("result_name", "mcs_buildings_samples")

In [ ]:
mc.run_analysis()

In [ ]:
building_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
df_building_dmg = building_damage_mcs_samples.get_dataframe_from_csv()
# Display top 5 rows of output data
df_building_dmg.head()

## Chaining with Joplin Electrical Power Facility

### Substations Damage

In [ ]:
epf_substations_dmg = EpfDamage(client)

epf_sub_mapping_id = "5d8a326fb9219c068ea798e7" # Create a mapping to assign tornado fragilities to substations.
fragility_service = FragilityService(client)    # loading fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping(epf_sub_mapping_id))
epf_substations_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

epf_substations_id = "5d92355bb9219c06ae7e386a"

epf_substations_dmg.load_remote_input_dataset("epfs", epf_substations_id)
result_name = "Joplin_epf_substations_dmg_result"
epf_substations_dmg.set_parameter("result_name", result_name)
epf_substations_dmg.set_parameter("hazard_type", hazard_type)
epf_substations_dmg.set_parameter("hazard_id", hazard_id)
epf_substations_dmg.set_parameter("num_cpu", 4)
epf_substations_dmg.set_parameter("fragility_key", "substations")

In [ ]:
epf_substations_dmg.run_analysis()

### MC substations failure probability

In [ ]:
substation_dmg_result = epf_substations_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", substation_dmg_result)
mc.set_parameter("num_cpu", 4)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["DS_0","DS_1","DS_2","DS_3","DS_4"])
mc.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3", "DS_4"])

mc.set_parameter("result_name", "mcs_substations_samples")

In [ ]:
mc.run_analysis()

In [ ]:
substations_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
df_sub_dmg = substations_damage_mcs_samples.get_dataframe_from_csv()
df_sub_dmg.head()

### Poles Damage

In [ ]:
epf_poles_id = "5d923daab9219c06ae84afb0"  # Load the Joplin poles dataset, which is a package of GIS files.

epf_poles_mapping_id = "5d8a326fb9219c068ea798e7"  # Create a mapping to assign tornado fragilities to poles.

epf_poles_dmg = EpfDamage(client)

epf_poles_dmg.load_remote_input_dataset("epfs", epf_poles_id)

fragility_service = FragilityService(client)        # loading fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping(epf_poles_mapping_id))
epf_poles_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

result_name = "Joplin_epf_poles_dmg_result"
epf_poles_dmg.set_parameter("result_name", result_name)
epf_poles_dmg.set_parameter("hazard_type", hazard_type)
epf_poles_dmg.set_parameter("hazard_id", hazard_id)
epf_poles_dmg.set_parameter("num_cpu", 4)
epf_poles_dmg.set_parameter("fragility_key", "poles")

In [ ]:
epf_poles_dmg.run_analysis()

### MC poles failure probability

In [ ]:
pole_dmg_result = epf_poles_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", pole_dmg_result)
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["DS_0","DS_1","DS_2","DS_3","DS_4"])
mc.set_parameter("failure_state_keys", ["DS_1","DS_2","DS_3","DS_4"])

mc.set_parameter("result_name", "mcs_poles_samples")

In [ ]:
mc.run_analysis()

In [ ]:
poles_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
df_poles_dmg = poles_damage_mcs_samples.get_dataframe_from_csv()
df_poles_dmg.head()

## Chaining with building functionality analysis

In [ ]:
bldg_func = BuildingFunctionality(client)
    
bldg_func.set_input_dataset("building_damage_mcs_samples", building_damage_mcs_samples)
bldg_func.set_input_dataset("substations_damage_mcs_samples", substations_damage_mcs_samples)
bldg_func.set_input_dataset("poles_damage_mcs_samples", poles_damage_mcs_samples)
bldg_func.load_remote_input_dataset("interdependency_dictionary", "5dcf4a34b9219ca5e4118312")

bldg_func.set_parameter("result_name", "mcs_functionality_probability")

In [ ]:
bldg_func.run_analysis()

In [ ]:
bldg_funct = bldg_func.get_output_dataset('functionality_samples')
df_building_func = bldg_funct.get_dataframe_from_csv()
df_building_func.head()

In [ ]:
bldg_prob = bldg_func.get_output_dataset('functionality_probability')
df_building_prob = bldg_prob.get_dataframe_from_csv()
df_building_prob.head()

## Physical Service Resilience Metrics

After finding the damage level for each component (buildings, substations, and poles) based on the components' fragility curves, their intrinsic failure status is expressed as a binary format with either failed (0) or not-failed (1). Then the failure status of all the buildings is updated by considering their dependencies with the corresponding electric power facilities.

Calculate and show all three infrastructure damages and resulting total building funcionality.

In [ ]:
import numpy as np
import pandas as pd

# buildings damage
bdm = df_building_dmg["failure"].str.split(pat=",")
# strings to integers, change fail to 1
bdm = 1 - bdm.apply(pd.to_numeric)
# convert panda frame to numpy ndarray and do the statistics
bdm = bdm.to_numpy()
# convert list of np arrays into single one
bdm = np.stack(bdm)
# percent
bdm_guid = 100 * bdm.mean(axis=0)
bdm_mean = np.mean(bdm_guid)
bdm_std = np.std(bdm_guid)

# substations damage
sdm = df_sub_dmg["failure"].str.split(pat=",")
sdm = 1 - sdm.apply(pd.to_numeric)
sdm = sdm.to_numpy()
sdm = np.stack(sdm)
sdm_guid = 100 * sdm.mean(axis=0)
sdm_mean = np.mean(sdm_guid)
sdm_std = np.std(sdm_guid)

# poles damage
pdm = df_poles_dmg["failure"].str.split(pat=",")
pdm = 1 - pdm.apply(pd.to_numeric)
pdm = pdm.to_numpy()
pdm = np.stack(pdm)
pdm_guid = 100 * pdm.mean(axis=0)
pdm_mean = np.mean(pdm_guid)
pdm_std = np.std(pdm_guid)

# buildings combined functionality
bfm = df_building_func["samples"].str.split(pat=",")
bfm = 1 - bfm.apply(pd.to_numeric)
bfm = bfm.to_numpy()
bfm = np.stack(bfm)
bfm_guid = 100 * bfm.mean(axis=0)
bfm_mean = np.mean(bfm_guid)
bfm_std = np.std(bfm_guid)

In [ ]:
# dataframe from statistical results
res_data = {"infrastructure": ["Building nonfunctional", "Building damage", "Substation damage", "Pole damage"],
            "percentage": [np.round(bfm_mean, 2), np.round(bdm_mean, 2), 
                           np.round(sdm_mean, 2), np.round(pdm_mean, 2)],
            "std": [np.round(bfm_std, 2), np.round(bdm_std, 2), 
                    np.round(sdm_std, 2), np.round(pdm_std, 2)]}
func_data = pd.DataFrame(res_data)

In [ ]:
print(func_data)

In [ ]:
## Ch